In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import tensorflow as tf

In [2]:
chord_dataset = pd.read_csv('chord_data.csv')

In [3]:
def remove_special_characters(text):
    return text.replace('[', '').replace(']', '').replace(',', '')

for i in range(len(chord_dataset)):
    chord_dataset['progression'].iloc[i] = remove_special_characters(chord_dataset['progression'].iloc[i])

Streaming output truncated to the last 5000 lines.
<ipython-input-3-c7cf58b64455>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chord_dataset['progression'].iloc[i] = remove_special_characters(chord_dataset['progression'].iloc[i])
<ipython-input-3-c7cf58b64455>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chord_dataset['progression'].iloc[i] = remove_special_characters(chord_dataset['progression'].iloc[i])
<ipython-input-3-c7cf58b64455>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [19]:
chord_dataset.to_csv('chord_data.csv')

In [6]:
chord_dataset

,genres,progression,start_key
0,pop,"['A', 'C#m', 'B']",F#m
1,pop,"['Bm', 'E', 'F#m']",D
2,pop,"['Em', 'Am', 'G']",C
3,pop,"['G', 'F', 'Dm']",Am
4,pop,"['G', 'C9', 'G']",C9
...,...,...,...
7707,rock,"['G', 'D', 'Em']",Am
7708,rock,"['C', 'G', 'D5']",G
7709,rock,"['E', 'A', 'F#']",B
7710,rock,"['G', 'C', 'Am']",D


In [7]:
from collections import Counter

# Initialize vocabulary
vocab = {'<pad>': 0, '<unk>': 1}  # Start with special tokens

# Iterate through columns and add unique tokens to vocabulary
for column in ['start_key', 'progression','genres']:
    for text in chord_dataset[column]:
        tokens = text.split()  # Tokenize text
        for token in tokens:
            if token not in vocab:
                vocab[token] = len(vocab)  # Add token to vocabulary

print("Vocabulary size:", len(vocab))

Vocabulary size: 1466


In [8]:
vocab

{'<pad>': 0,
 '<unk>': 1,
 'F#m': 2,
 'D': 3,
 'C': 4,
 'Am': 5,
 'C9': 6,
 'Bm': 7,
 'C#m': 8,
 'E': 9,
 'G': 10,
 'F7M': 11,
 'F#': 12,
 'Ab': 13,
 'Bb/D': 14,
 'Em': 15,
 'A': 16,
 'D7M': 17,
 'Cadd9': 18,
 'F': 19,
 'Bb': 20,
 'Dm': 21,
 'G#m': 22,
 'A5': 23,
 'A/D': 24,
 'Cm': 25,
 'Gm': 26,
 'C#': 27,
 'Em7': 28,
 'Fm': 29,
 'Am7': 30,
 'B': 31,
 'Esus4': 32,
 'G#': 33,
 'C#madd9': 34,
 'Bm/F#': 35,
 'Dsus4/G': 36,
 'C7M': 37,
 'D9': 38,
 'Eb': 39,
 'C7': 40,
 'Dmaj7': 41,
 'G9': 42,
 'Abm': 43,
 'B7': 44,
 'D7': 45,
 'F#7': 46,
 'Cmaj7': 47,
 'Db': 48,
 'D#m': 49,
 'Fmaj7': 50,
 'A#': 51,
 'Gm7': 52,
 'C/G': 53,
 'Cm7': 54,
 'G5/E': 55,
 'Bb7': 56,
 'Ebm': 57,
 'Dm7': 58,
 'G#maj7': 59,
 'F/G': 60,
 'Bm7': 61,
 'C7/9': 62,
 'Gsus4': 63,
 'Bbmaj7': 64,
 'C#m7': 65,
 'E5': 66,
 'F5': 67,
 'Asus4': 68,
 'Fmaj7/A': 69,
 'A/C#': 70,
 'D5': 71,
 'Em9': 72,
 'A9': 73,
 'G7M': 74,
 'Dbm': 75,
 'A7M': 76,
 'Gdim': 77,
 'G7': 78,
 'Bbm': 79,
 'A6': 80,
 'G5': 81,
 'Dsus2': 82,
 'Gb': 83,


In [9]:
chord_dataset['genres'].unique()

array(['pop', 'k-pop', 'rap', 'canadian pop', 'reggaeton', 'hip hop',
       'alternative r&b', 'australian hip', 'country rap', 'rock',
       'permanent wave', 'r&b', 'chicago drill', 'ohio hip', 'emo rap',
       'metal', 'dance pop', 'cali rap', 'modern rock', 'pop r&b',
       'desi pop', 'bachata latin', 'electro house', 'rock metal',
       'house', 'soul pop', 'pop house', 'country', 'australian psych',
       'r&b pop', 'funk pop', 'pop rap', 'alternative hip',
       'irish singer-songwriter', 'metalcore', 'vancouver indie',
       'speedrun video', 'alt z', 'boy band', 'neo mellow', 'rock pop',
       'latin', 'beatlesque britpop', 'indie', 'brostep complextro',
       'pop indie', 'folk-pop modern', 'dutch pop', 'disco mellow',
       'canadian hip', 'latin pop', 'soul', 'punk', 'adult standards',
       'lo-fi chill', 'trance', 'pop rock', 'electropop modern',
       'brazilian edm', 'wave', 'ccm worship', 'indie poptimism',
       'edm pop', 'modern blues', 'sad rap', 'ke

In [11]:
# from sklearn.model_selection import train_test_split
# train_data, val_data = train_test_split(chord_dataset,test_size=0.2, random_state=42)

In [12]:
# train_data.reset_index(drop=True)
# val_data.reset_index(drop=True)

In [3]:
chord_dataset['progression']

0        ['A', 'C#m', 'B']
1       ['Bm', 'E', 'F#m']
2        ['Em', 'Am', 'G']
3         ['G', 'F', 'Dm']
4         ['G', 'C9', 'G']
               ...        
7707      ['G', 'D', 'Em']
7708      ['C', 'G', 'D5']
7709      ['E', 'A', 'F#']
7710      ['G', 'C', 'Am']
7711       ['E', 'A', 'E']
Name: progression, Length: 7712, dtype: object

In [4]:
for i in range(len(chord_dataset)):
    chord_dataset['progression'].iloc[i] = chord_dataset['progression'].iloc[i].replace('[', '').replace(']', '')

In [5]:
chord_dataset['progression']

0        'A', 'C#m', 'B'
1       'Bm', 'E', 'F#m'
2        'Em', 'Am', 'G'
3         'G', 'F', 'Dm'
4         'G', 'C9', 'G'
              ...       
7707      'G', 'D', 'Em'
7708      'C', 'G', 'D5'
7709      'E', 'A', 'F#'
7710      'G', 'C', 'Am'
7711       'E', 'A', 'E'
Name: progression, Length: 7712, dtype: object

In [22]:
from sklearn.preprocessing import LabelEncoder

# Example genre data
genres = chord_dataset['genres']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform genres to integer labels
encoded_labels = label_encoder.fit_transform(genres)

# Print the generated labels
print("Generated Labels:", encoded_labels)

Generated Labels: [341 341 341 ... 382 382 382]


In [21]:
from collections import Counter

# Initialize vocabulary
vocab_chords = {'<pad>': 0, '<unk>': 1}  # Start with special tokens

# Iterate through columns and add unique tokens to vocabulary
for column in ['start_key', 'progression']:
    for text in chord_dataset[column]:
        tokens = text.split()  # Tokenize text
        for token in tokens:
            if token not in vocab_chords:
                vocab_chords[token] = len(vocab_chords)  # Add token to vocabulary

print("Vocabulary size:", len(vocab_chords))

Vocabulary size: 987


In [ ]:
vocab_genres_file = "vocab_genres.txt"

# Write the vocab list to the file
with open(vocab_genres_file, "w") as f:
    for word in vocab_genres:
        f.write(word + "\n")

print("Vocabulary saved to", vocab_genres_file)

Vocabulary saved to vocab_genres.txt


In [24]:
vocab_chords

{'<pad>': 0,
 '<unk>': 1,
 'F#m': 2,
 'D': 3,
 'C': 4,
 'Am': 5,
 'C9': 6,
 'Bm': 7,
 'C#m': 8,
 'E': 9,
 'G': 10,
 'F7M': 11,
 'F#': 12,
 'Ab': 13,
 'Bb/D': 14,
 'Em': 15,
 'A': 16,
 'D7M': 17,
 'Cadd9': 18,
 'F': 19,
 'Bb': 20,
 'Dm': 21,
 'G#m': 22,
 'A5': 23,
 'A/D': 24,
 'Cm': 25,
 'Gm': 26,
 'C#': 27,
 'Em7': 28,
 'Fm': 29,
 'Am7': 30,
 'B': 31,
 'Esus4': 32,
 'G#': 33,
 'C#madd9': 34,
 'Bm/F#': 35,
 'Dsus4/G': 36,
 'C7M': 37,
 'D9': 38,
 'Eb': 39,
 'C7': 40,
 'Dmaj7': 41,
 'G9': 42,
 'Abm': 43,
 'B7': 44,
 'D7': 45,
 'F#7': 46,
 'Cmaj7': 47,
 'Db': 48,
 'D#m': 49,
 'Fmaj7': 50,
 'A#': 51,
 'Gm7': 52,
 'C/G': 53,
 'Cm7': 54,
 'G5/E': 55,
 'Bb7': 56,
 'Ebm': 57,
 'Dm7': 58,
 'G#maj7': 59,
 'F/G': 60,
 'Bm7': 61,
 'C7/9': 62,
 'Gsus4': 63,
 'Bbmaj7': 64,
 'C#m7': 65,
 'E5': 66,
 'F5': 67,
 'Asus4': 68,
 'Fmaj7/A': 69,
 'A/C#': 70,
 'D5': 71,
 'Em9': 72,
 'A9': 73,
 'G7M': 74,
 'Dbm': 75,
 'A7M': 76,
 'Gdim': 77,
 'G7': 78,
 'Bbm': 79,
 'A6': 80,
 'G5': 81,
 'Dsus2': 82,
 'Gb': 83,


In [26]:
vocab_keys = vocab_chords.keys()

# Convert keys to a list
vocab_list = list(vocab_keys)

print(vocab_list[0:])

['<pad>', '<unk>', 'F#m', 'D', 'C', 'Am', 'C9', 'Bm', 'C#m', 'E', 'G', 'F7M', 'F#', 'Ab', 'Bb/D', 'Em', 'A', 'D7M', 'Cadd9', 'F', 'Bb', 'Dm', 'G#m', 'A5', 'A/D', 'Cm', 'Gm', 'C#', 'Em7', 'Fm', 'Am7', 'B', 'Esus4', 'G#', 'C#madd9', 'Bm/F#', 'Dsus4/G', 'C7M', 'D9', 'Eb', 'C7', 'Dmaj7', 'G9', 'Abm', 'B7', 'D7', 'F#7', 'Cmaj7', 'Db', 'D#m', 'Fmaj7', 'A#', 'Gm7', 'C/G', 'Cm7', 'G5/E', 'Bb7', 'Ebm', 'Dm7', 'G#maj7', 'F/G', 'Bm7', 'C7/9', 'Gsus4', 'Bbmaj7', 'C#m7', 'E5', 'F5', 'Asus4', 'Fmaj7/A', 'A/C#', 'D5', 'Em9', 'A9', 'G7M', 'Dbm', 'A7M', 'Gdim', 'G7', 'Bbm', 'A6', 'G5', 'Dsus2', 'Gb', 'C/E', 'Am/E', 'A#m', 'A7sus4', 'A#5', 'Am9', 'Fsus2', 'A4/7', 'C5', 'C#maj7', 'Eb7M9', 'b', 'Dsus2/G', 'C#sus9', 'Asus2', 'Gmaj7', 'B5', 'Bm5/7', 'D6', 'F#5', 'C#4', 'Bsus2', 'G#5', 'F7', 'D/A', 'Eb5', 'Dsus4', 'E7', 'F/C', 'D#', 'F#m9', 'Gm7/D', 'Abmaj9/C', 'Gadd9', 'C#5', 'Bm/E', 'F9', 'Gbmaj7', 'bm', 'F#m/A', 'Gbm', 'F#m7', 'Abmaj7', 'Db7', 'C/A', 'Bbdim7', 'E9', 'C11', 'G/B', 'Amaj9', 'Db/F', 'G#m7', 

In [7]:
from collections import Counter

# Initialize vocabulary
vocab_genres= {'<pad>': 0, '<unk>': 1}  # Start with special tokens

# Iterate through columns and add unique tokens to vocabulary
for column in ['genres']:
    for text in chord_dataset[column]:
        tokens = text.split()  # Tokenize text
        for token in tokens:
            if token not in vocab_genres:
                vocab_genres[token] = len(vocab_genres)  # Add token to vocabulary

print("Vocabulary size:", len(vocab_genres))

Vocabulary size: 295


In [8]:
vocab_genres

{'<pad>': 0,
 '<unk>': 1,
 'pop': 2,
 'k-pop': 3,
 'rap': 4,
 'canadian': 5,
 'reggaeton': 6,
 'hip': 7,
 'hop': 8,
 'alternative': 9,
 'r&b': 10,
 'australian': 11,
 'country': 12,
 'rock': 13,
 'permanent': 14,
 'wave': 15,
 'chicago': 16,
 'drill': 17,
 'ohio': 18,
 'emo': 19,
 'metal': 20,
 'dance': 21,
 'cali': 22,
 'modern': 23,
 'desi': 24,
 'bachata': 25,
 'latin': 26,
 'electro': 27,
 'house': 28,
 'soul': 29,
 'psych': 30,
 'funk': 31,
 'irish': 32,
 'singer-songwriter': 33,
 'metalcore': 34,
 'vancouver': 35,
 'indie': 36,
 'speedrun': 37,
 'video': 38,
 'alt': 39,
 'z': 40,
 'boy': 41,
 'band': 42,
 'neo': 43,
 'mellow': 44,
 'beatlesque': 45,
 'britpop': 46,
 'brostep': 47,
 'complextro': 48,
 'folk-pop': 49,
 'dutch': 50,
 'disco': 51,
 'punk': 52,
 'adult': 53,
 'standards': 54,
 'lo-fi': 55,
 'chill': 56,
 'trance': 57,
 'electropop': 58,
 'brazilian': 59,
 'edm': 60,
 'ccm': 61,
 'worship': 62,
 'poptimism': 63,
 'blues': 64,
 'sad': 65,
 'kentucky': 66,
 'new': 67,
 '

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Modify the TextDataset class to include genre indices
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, chords, genres, vocab_chords,vocab_genres, seq_length):
        self.texts = texts
        self.chords = chords
        self.genres = genres
        self.vocab_chords = vocab_chords
        self.vocab_genres = vocab_genres
        self.seq_length = seq_length

    def __len__(self):
        return len(self.texts) - self.seq_length

    def __getitem__(self, idx):
        text_chunk = self.texts[idx:idx+self.seq_length]
        chord_chunk = self.chords[idx+self.seq_length]
        input_seq = [self.vocab_chords.get(token, self.vocab_chords['<unk>']) for token in text_chunk]

        # Create a zero tensor for the target chord sequence
        target_onehot_seq = torch.zeros(len(chord_chunk.split()), len(self.vocab_chords))

        # Convert target chord sequence to one-hot encoded vectors
        for i, token in enumerate(chord_chunk.split()):
            index = self.vocab_chords.get(token, self.vocab_chords['<unk>'])
            target_onehot_seq[i, index] = 1

        genre = self.genres[idx:idx+self.seq_length]
        genre_seq = [self.vocab_genres.get(token, self.vocab_genres['<unk>']) for token in genre]
        return torch.tensor(input_seq), torch.tensor(target_onehot_seq), torch.tensor(genre_seq)


In [11]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_genres):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)  # Adjust output size
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.genre_embedding = nn.Embedding(num_genres, hidden_dim)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def forward(self, x):
        batch_size = x.size(0)  # Get batch size dynamically
        hidden = self.init_hidden(batch_size)  # Initialize hidden state

        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)

        # Initialize list to store outputs
        outputs = []

        # Iterate three times to generate three letters
        for i in range(3):
            # Get output at current time step
            current_output = self.fc(output[:, i, :])

            # Apply softmax activation to get probabilities
            output_probs = torch.softmax(current_output, dim=-1)

            # Convert probabilities to one-hot encoded vectors
            # _, max_indices = torch.max(output_probs, dim=-1)
            # one_hot_vectors = torch.eye(output_probs.size(-1))[max_indices]

            # Append one-hot vector to outputs list
            outputs.append(output_probs)

        # Stack outputs along time step dimension
        outputs = torch.stack(outputs, dim=1)

        return outputs

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(self.device),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(self.device))


In [54]:
import torch.distributions as dist

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for inputs, targets, genres in train_loader:
        inputs, targets, genres = inputs.to(device), targets.to(device), genres.to(device)
        optimizer.zero_grad()
        hidden = model.init_hidden(inputs.size(0))

        # Concatenate inputs with genre information
        inputs = torch.cat((inputs, genres), dim=1)

        outputs = model(inputs)

        # print(outputs.shape)
        # print(targets.shape)

        outputs = outputs.view(-1)
        targets = targets.view(-1)
        loss = F.cross_entropy(outputs, targets).requires_grad_(True)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def generate_text(model, start_text, genres, vocab_chords,vocab_genres, max_length=100, temperature=1.0, device='cpu'):
    model.eval()
    with torch.no_grad():
        # Create a TextDataset_gen_test instance for the start text and genre
        test_val = TextDataset_gen_test(start_text, genres, vocab_chords,vocab_genres)
        input_tensor, genre_tensor = test_val[0]
        input_tensor = input_tensor.to(device)
        genre_tensor = genre_tensor.to(device)

        # Concatenate input tensor with genre tensor
        inputs = torch.cat((input_tensor.view(1, 1), genre_tensor.unsqueeze(0)), dim=1)

        # Generate output from the model
        output = model(inputs)

        output_reshaped = output.view(-1,len(vocab_chords))

        # # Convert one-hot encoded vectors to probability distribution by applying softmax
        probs = torch.softmax(output_reshaped, dim=-1)

        # Create a Categorical distribution from the probability distribution
        categorical = dist.Categorical(probs)

        # Sample a token index from the categorical distribution
        sampled_token_idx = categorical.sample()
        generated_text = detokenize_text(sampled_token_idx.tolist(), vocab_chords)

        return generated_text


In [41]:
# Sample parameters
vocab_size = len(vocab_chords)
num_genres = len(vocab_genres)
embedding_dim = 512
hidden_dim = 2048
num_layers = 6
seq_length = 8
num_epochs = 10
learning_rate = 0.1
batch_size = 64
max_length = 100
temperature = 0.8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize model, optimizer, and loss function
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, num_genres).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Sample dataset creation (replace with your actual dataset)
dataset = TextDataset(chord_dataset['start_key'],chord_dataset['progression'],chord_dataset['genres'],vocab_chords,vocab_genres,seq_length)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    loss = train(model, train_loader, optimizer, criterion, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')


<ipython-input-9-01e1792d314f>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(input_seq), torch.tensor(target_onehot_seq), torch.tensor(genre_seq)


Epoch [1/10], Loss: 2307.3406
Epoch [2/10], Loss: 2307.3011
Epoch [3/10], Loss: 2307.3012
Epoch [4/10], Loss: 2307.3012
Epoch [5/10], Loss: 2307.3012
Epoch [6/10], Loss: 2307.3012
Epoch [7/10], Loss: 2307.3012
Epoch [8/10], Loss: 2307.3012
Epoch [9/10], Loss: 2307.3011
Epoch [10/10], Loss: 2307.3011


In [42]:
# Assuming your model is named 'model'
torch.save(model.state_dict(), 'lstm_model.pth')

In [55]:
class TextDataset_gen_test():
    def __init__(self, text, genre, vocab_chord,vocab_genres):
        self.text = text
        self.genre = genre
        self.vocab_chords = vocab_chords
        self.vocab_genres = vocab_genres

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        input_seq = [self.vocab_chords.get(token, self.vocab_chords['<unk>']) for token in self.text]
        genre_seq = [self.vocab_genres.get(token, self.vocab_genres['<unk>']) for token in self.genre]
        return torch.tensor(input_seq), torch.tensor(genre_seq)


In [56]:
def detokenize_text(tokens, vocab, unknown_token="<unk>"):
    """Detokenize a list of token values using a vocabulary dictionary."""
    reversed_vocab = {v: k for k, v in vocab.items()}
    return ' '.join([reversed_vocab.get(token, unknown_token) for token in tokens])

In [57]:
start_text = "C"
genres = "rock"  # Example genre
model = model.to('cuda')
generated_text = generate_text(model, start_text, genres, vocab_chords,vocab_genres, temperature=0.5, device='cuda')
print("Generated text:", generated_text)

Generated text: 'B4/C#', 'Em7/B', 'D6',
